In [ ]:
# MMDetection 3.x (tested with 3.3.0)
from mmengine.config import Config
from mmengine.runner import Runner
import os

# --- 1) Paths you set ---
CONFIG ='C:/Users/heheh/mmdetection/configs/mask_rcnn/mask-rcnn_r50_fpn_1x_coco.py'
CHECKPOINT = 'C:/Users/heheh/mmdetection/checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'
DATA_ROOT = 'C:/Users/heheh/mmdetection/data/coco'
ANN_FILE  = os.path.join(DATA_ROOT, 'annotations', 'instances_val2017.json')
IMG_PREFIX = os.path.join(DATA_ROOT, 'instance_maskrcnn_adv')
# --- 2) Load config and override dataset/evaluator/paths ---
cfg = Config.fromfile(CONFIG)
cfg.default_scope = 'mmdet'
cfg.load_from = CHECKPOINT
cfg.work_dir = './work_dirs/eval_maskrcnn_coco'

# Dataloader overrides (batch size/workers as you wish)
cfg.test_dataloader.batch_size = 2
cfg.test_dataloader.num_workers = 2

# Point the test dataloader to your COCO images + annotation file
# (keeps the original test pipeline from the config)
test_dataset = cfg.test_dataloader.dataset
test_dataset.type = 'CocoDataset'
test_dataset.data_root = DATA_ROOT
test_dataset.ann_file = ANN_FILE
test_dataset.data_prefix = dict(img=IMG_PREFIX)
test_dataset.test_mode = True
test_dataset.indices = list(range(10))

# Optional: evaluate on a small subset first (uncomment to limit to N images)
# test_dataset.indices = list(range(200))  # evaluate only first 200 images

# COCO metric (bbox + segm). You’ll get mAP, AP50, AP75, per-area, etc.
cfg.test_evaluator = dict(
    type='CocoMetric',
    ann_file=ANN_FILE,
    metric=['bbox', 'segm'],
    format_only=False
)

# If your config uses visualizer, disable saving images during test
if hasattr(cfg, 'visualizer'):
    cfg.visualizer.vis_backends = None

# --- 3) Run evaluation ---
runner = Runner.from_cfg(cfg)
runner.test()
